In [1]:
import pandas as pd
import numpy as np
import itertools
from pulp import *

In [2]:
problem = LpProblem("E-Scooter Allocation", LpMinimize)

In [3]:
df_distance = pd.read_csv("Model Data - Distance Matrix.csv")
df_demand = pd.read_csv("Model Data - E-scooter Demand.csv")

In [4]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))

In [5]:
df_distance.fillna(0, inplace=True)

distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [6]:
demand = df_demand.values[:,1:]
# demand[i][j][t] - the demand at starting time t from location i to location j
demand_dict = {(x,y,0): demand[x][y] for x in range(demand.shape[0]) for y in range(demand.shape[1])}
demand_dict.update({(x,y,1): demand[y][x] for x in range(demand.shape[1]) for y in range(demand.shape[0])})
demand_dict.update({(x,y,2): 0 for x in range(demand.shape[0]) for y in range(demand.shape[1])})

In [7]:
# Parameters
M = sys.maxsize
N = 20
# Cost values
C_scooter_km = 42
C_mf_scooter = 5
C_mf_dock = 14
# Penalty value 
C_pen = 1000

In [8]:
# df_penalty_cost = pd.read_csv("Penalty Carbon Costs Costs.csv")
# df_penalty_cost
# df_undemand_prob = pd.read_csv("Unmet Demand Displacement Prob.csv")
# df_undemand_prob.head()

In [9]:
# Sets
# time points
T = [0,1,2]
# each location at a given time
X = list(itertools.product(location_idx, T))
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
A2 = [(xi, xj) for xi in X for xj in X if xi[0]==xj[0] and xi[1]+1==xj[1]]
# Relocation
A3 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]
#location_pairs = list(itertools.permutations(locations,2))

In [10]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat=const.LpInteger)
# Relocation
# Rij = LpVariable.dicts("#Scooters", A3, 0, cat=const.LpInteger)
Vit = LpVariable.dicts("#Available_scooters",X,0,cat=const.LpInteger)
Sit = LpVariable.dicts("#Stocked_Scooters", A2, 0, cat=const.LpInteger)
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat=const.LpInteger)

In [11]:
# Objective function
problem+=lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*C_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[((i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen for ((i,ti),(j,tj)) in A1) + \
    lpSum(Vit[(i,t)]*C_mf_scooter for (i,t) in X if t==0) + \
    lpSum(Yi[i]*C_mf_dock for i in location_idx), "Objective function"

In [13]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,tj))] for (j,tj) in X if tj==ti and i !=j) + lpSum(Ditj[((j, tj),(i, ti))] for (j,tj) in X if tj == ti-1 and i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [14]:
for (i,ti) in X:
    if ti!=2:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,tj))]for (j,tj) in X if i !=j and ti+1==tj) == Sit[((i,ti), (i,ti+1))], f"Stocked Scooters for {(i,ti)}"

In [15]:
# Simplified relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])], f"Daily balance for location {i}"

In [16]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [17]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= M*Yi[i], f"Availability against Dock present{(i,ti)}"

In [18]:
for i in location_idx:
    problem+=Yi[i] <= Zi[i], f"Size constraint for {i}"

In [19]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [20]:
problem+=lpSum(Yi[i] for i in location_idx) == N, f" Number of docks"

In [21]:
# problem.writeLP("E-scooterProblem.lp")

In [22]:
problem.solve()

1

In [23]:
LpStatus[problem.status]

'Optimal'

In [24]:
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Size_0 = 1.0
Size_1 = 1.0
Size_10 = 1.0
Size_100 = 1.0
Size_101 = 1.0
Size_102 = 1.0
Size_103 = 1.0
Size_104 = 1.0
Size_105 = 1.0
Size_106 = 1.0
Size_107 = 1.0
Size_108 = 1.0
Size_109 = 1.0
Size_11 = 1.0
Size_110 = 1.0
Size_12 = 1.0
Size_13 = 1.0
Size_14 = 1.0
Size_15 = 1.0
Size_16 = 1.0
Station_Presence_0 = 1.0
Station_Presence_1 = 1.0
Station_Presence_10 = 1.0
Station_Presence_100 = 1.0
Station_Presence_101 = 1.0
Station_Presence_102 = 1.0
Station_Presence_103 = 1.0
Station_Presence_104 = 1.0
Station_Presence_105 = 1.0
Station_Presence_106 = 1.0
Station_Presence_107 = 1.0
Station_Presence_108 = 1.0
Station_Presence_109 = 1.0
Station_Presence_11 = 1.0
Station_Presence_110 = 1.0
Station_Presence_12 = 1.0
Station_Presence_13 = 1.0
Station_Presence_14 = 1.0
Station_Presence_15 = 1.0
Station_Presence_16 = 1.0


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [30]:
value(problem.objective)

In [25]:
f = open("obj.txt", "w+")
print(problem.objective, file=f)
f.close()